In [ ]:
!set -x \
&& pip install konlpy \
&& curl -s https://raw.githubusercontent.com/konlpy/konlpy/master/scripts/mecab.sh | bash -x

from google.colab import drive
drive.mount('/content/drive')

+ pip install konlpy
     |████████████████████████████████| 19.4MB 5.1MB/s 
     |████████████████████████████████| 460kB 45.2MB/s 
     |████████████████████████████████| 92kB 12.1MB/s 
  Found existing installation: beautifulsoup4 4.6.3
    Uninstalling beautifulsoup4-4.6.3:
      Successfully uninstalled beautifulsoup4-4.6.3
+ bash -x
+ curl -s https://raw.githubusercontent.com/konlpy/konlpy/master/scripts/mecab.sh
+ mecab_dicdir=/usr/local/lib/mecab/dic/mecab-ko-dic
+ set -e
++ uname
+ os=Linux
+ [[ ! Linux == \L\i\n\u\x ]]
+ hash sudo
+ sudo=sudo
+ python=python3
+ hash pyenv
+ at_user_site=
++ check_python_site_location_is_writable
++ python3 -
+ [[ 1 == \0 ]]
+ hash automake
+ echo 'Installing automake (A dependency for mecab-ko)'
Installing automake (A dependency for mecab-ko)
+ install_automake
+ '[' Linux == Linux ']'
++ grep -Ei 'debian|buntu|mint' /etc/lsb-release /etc/os-release
+ '[' '/etc/lsb-release:DISTRIB_ID=Ubuntu
/etc/lsb-release:DISTRIB_DESCRIPTION="Ubuntu 18.04.5

In [ ]:
import os
os.chdir('/content/drive/MyDrive/Colab Notebooks')

!git clone https://github.com/lovit/textrank.git
os.chdir("textrank")
!python setup.py install

!pip install gensim
!pip install kmeans_pytorch

fatal: destination path 'textrank' already exists and is not an empty directory.
running install
running bdist_egg
running egg_info
writing textrank.egg-info/PKG-INFO
writing dependency_links to textrank.egg-info/dependency_links.txt
writing top-level names to textrank.egg-info/top_level.txt
reading manifest file 'textrank.egg-info/SOURCES.txt'
writing manifest file 'textrank.egg-info/SOURCES.txt'
installing library code to build/bdist.linux-x86_64/egg
running install_lib
running build_py
creating build/bdist.linux-x86_64/egg
creating build/bdist.linux-x86_64/egg/textrank
copying build/lib/textrank/word.py -> build/bdist.linux-x86_64/egg/textrank
copying build/lib/textrank/rank.py -> build/bdist.linux-x86_64/egg/textrank
copying build/lib/textrank/summarizer.py -> build/bdist.linux-x86_64/egg/textrank
copying build/lib/textrank/sentence.py -> build/bdist.linux-x86_64/egg/textrank
copying build/lib/textrank/utils.py -> build/bdist.linux-x86_64/egg/textrank
copying build/lib/textrank/__i

In [ ]:
import numpy as np
import pandas as pd
from konlpy.tag import Mecab
from gensim.models import Word2Vec
from textrank import KeywordSummarizer
import operator
from gensim.models import FastText
import torch
from kmeans_pytorch import kmeans
from sklearn.cluster import KMeans

In [ ]:
def category_based_counting(cluster_keyword_result):
    item_list = []
    for cluster in cluster_keyword_result:
        item_dic = {}
        cnt = 0
        for item in cluster:
            if cnt == 10:
                break
            try:
                _item = item[0].split('/')[0]
                similar_item_list = embedding_model.most_similar(_item, topn=10)
                cnt+=1
                item_dic[_item] = item_dic.get(_item, 0) + 1
                for s_item in similar_item_list:
                    item_dic[s_item[0]] = item_dic.get(s_item[0], 0) + 1
            except:
                pass
        item_list.append(item_dic)

    sorted_item_top3 = []
    for item in item_list:
        sort_dict = sorted(item.items(), key=operator.itemgetter(1), reverse=True)
        sorted_item_top3.append(sort_dict[:3])
    return sorted_item_top3

def category_based_summation(cluster_keyword_result):
    item_list = []
    for cluster in cluster_keyword_result:
        item_dic = {}
        cnt = 0
        for item in cluster:
            if cnt == 10:
                break
            try:
                _item = item[0].split('/')[0]
                similar_item_list = embedding_model.most_similar(_item, topn=10)
                cnt+=1
                item_dic[_item] = item_dic.get(_item, 0) + 1
                for s_item in similar_item_list:
                    item_dic[s_item[0]] = item_dic.get(s_item[0], 0) + s_item[1]
            except:
                pass
        item_list.append(item_dic)

    sorted_item_top3 = []
    for item in item_list:
        sort_dict = sorted(item.items(), key=operator.itemgetter(1), reverse=True)
        sorted_item_top3.append(sort_dict[:3])
    return sorted_item_top3






In [ ]:
TEXT_REVIEW_COLUMN_NAME = 'text'
SENTENCE_IDX_COLUME_NAME = 'rawSentenceIdx'
DATA_PATH = '/content/drive/MyDrive/data/'
OUTPUT_PATH = '/content/drive/MyDrive/Model_savefile/'
mecab = Mecab()
models = [['Final_FastText', 17], ['Final_FastText', 8]]
#models = [['Word2Vec', 8], ['FastText', 8]]
def WordEmbedding_tokenizer(sents):
  word = []
  for sent in sents:
    words = mecab.pos(sent, join=True)
    words = [w.split('/')[0] for w in words if ('/NNG' in w or '/XR' in w or '/NNR' in w)]
    tt = []
    for w in words:
      if len(w) == 1:
        continue
      tt.append(w)
    word.append(tt)
  return word

def mecab_tokenizer(sent):
    words = mecab.pos(sent, join=True)
    words = [w for w in words if ('/NNG' in w or '/XR' in w or '/NNR' in w)]
    return words

def data_load(datafilename):
    traindatafile = DATA_PATH + datafilename + '.txt'
    df = pd.read_csv(traindatafile,encoding ='utf-8', sep='\t')
    df = df.dropna(axis=0)
    rawData = df[TEXT_REVIEW_COLUMN_NAME].values.tolist()
    rawSentenceIdx = df[SENTENCE_IDX_COLUME_NAME].values.tolist()
    tokenData = WordEmbedding_tokenizer(rawData)
    return rawData, tokenData, rawSentenceIdx

def Word2Vec_Embedding(tokenData):
    embedding_model = Word2Vec(tokenData, size=64, window = 3, min_count=5, workers=4, iter=20, sg=1)
    word_vectors = embedding_model.wv
    vocabs = word_vectors.vocab.keys()
    word_vectors_list = [word_vectors[v] for v in vocabs]
    return embedding_model, word_vectors, vocabs, word_vectors_list

def FastText_Embedding(tokenData):
    embedding_model = FastText(tokenData, size=64, window=3, min_count=5, workers=4, sg=1, iter=20)
    word_vectors = embedding_model.wv
    vocabs = word_vectors.vocab.keys()
    word_vectors_list = [word_vectors[v] for v in vocabs]
    return embedding_model, word_vectors, vocabs, word_vectors_list

def SentenceMapping(rawData, tokenData, word_vectors, _rawSentenceIdx):
    sentenceData = []
    preRawData = []
    rawSentenceIdx = []
    for idx, i in enumerate(tokenData):
        if len(i) == 0:
            continue
        sentenceData.append(i)
        preRawData.append(rawData[idx])
        rawSentenceIdx.append(_rawSentenceIdx[idx])

    sentenceToken = []
    for sentence in sentenceData:
        n = len(sentence)
        np_arr = np.zeros(shape = word_vectors["화장실"].shape)
        for word in sentence:
            try:
                np_arr += word_vectors[word]
            except:
                pass
        if np_arr.all() == np.zeros:
            continue
        sentenceToken.append(np_arr/n)
    return sentenceToken, preRawData, rawSentenceIdx

def k_means_cluster(sentenceToken, preRawData, rawSentenceIdx, K):
    estimator = KMeans(n_clusters=K, init='k-means++', random_state=42, algorithm='elkan')
    estimator = estimator.fit(sentenceToken)
    cluster_ids_x = estimator.labels_
    cluster_centers = estimator.cluster_centers_
    
    sentence_results = cluster_result_mapping(preRawData, cluster_ids_x, rawSentenceIdx, K)
    keyword_result = cluster_keyword_summarizer(sentence_results, K)

    return keyword_result, sentence_results, cluster_centers

def cluster_result_mapping(data, cluster_estimator, rawSentenceIdx, k):
    results = [[] for _ in range(k)]
    for item, sentenceIdx, label in zip(data, rawSentenceIdx, cluster_estimator):
        results[label].append([item, sentenceIdx])
    return results

def cluster_keyword_summarizer(results, k):
    keyword_extractor = KeywordSummarizer(
      tokenize = mecab_tokenizer,
      min_count=2,
      window=-1,                     # cooccurrence within a sentence
      min_cooccurrence=2,
      vocab_to_idx=None,             # you can specify vocabulary to build word graph
      df=0.85,                       # PageRank damping factor
      max_iter=30,                   # PageRank maximum iteration
      verbose=False
    )
    keyword_result = []
    for i in range(k):
        _tmp = keyword_extractor.summarize([item[0] for item in results[i]], topk=20)
        keyword_result.append(_tmp)
    return keyword_result


def cluster_based_word_counting_in_cluster(embedding_model, cluster_keyword_result, cluster_sentence_result):
    item_list = []
    for keyword, sentences in zip(cluster_keyword_result, cluster_sentence_result):
        item_dic = {}
        cnt = 0
        for item in keyword:
            if cnt == 10:
                break
            try:
                _item = item[0].split('/')[0]
                embedding_model.most_similar(_item, topn=10)
                cnt+=1
                for sentence in sentences:
                    item_dic[_item] = item_dic.get(_item, 0) + sentence[0].count(_item)
            except:
                pass
        item_list.append(item_dic)

    sorted_item_top3 = []
    for item in item_list:
        sort_dict = sorted(item.items(), key=operator.itemgetter(1), reverse=True)
        sorted_item_top3.append(sort_dict[:3])
    print(sorted_item_top3)
    return sorted_item_top3

def keyword_cluster_processing(models, datafilename):
    K = models[1]
    rawData, tokenData, rawSentenceIdx = data_load(datafilename)

    embedding_model, word_vectors, vocabs, word_vectors_list = FastText_Embedding(tokenData)

    sentenceToken, preRawData, rawSentenceIdx = SentenceMapping(rawData, tokenData, word_vectors, rawSentenceIdx)
    k_cluster_keyword_result, k_cluster_sentence_result, cluster_centroid = k_means_cluster(sentenceToken, preRawData, rawSentenceIdx, K)

    word_counting = cluster_based_word_counting_in_cluster(embedding_model, k_cluster_keyword_result, k_cluster_sentence_result)
    print(word_counting)

    top3Keyword = []   
    for cluster in word_counting:
        top3Keyword.append([item[0] for item in cluster[:3]])   

    word_counting = category_name_change(word_counting)
    print(word_counting)
    category_name = []
    item_set = set()
    equal_item = set()
    for cluster in word_counting:
        for item, score in cluster:
            if item == '호텔':
                item = '기타'
            if item == '칫솔' or item == '치약' or item == '수건' or item == '샴푸':
                item = '어메니티'
            category_name.append(item)
            break 

    

    save_model(models[0], K, embedding_model, category_name, cluster_centroid, top3Keyword)

def category_name_change(word_counting):
    while True:
        now_cluster = 0
        change = 0
        for i in range(len(word_counting)):
            if len(word_counting[i]) == 1:
                continue
            for j in range(i+1, len(word_counting)):
                if len(word_counting[j]) == 1:
                  continue
                if word_counting[j][0][0] == word_counting[i][0][0]:
                    change = 1
                    if word_counting[j][1][1] > word_counting[i][1][1]:
                        word_counting[j].pop(0)
                    else:
                        word_counting[i].pop(0)
        if change == 0:
            break
        
    return word_counting

def save_model(name, k, model, category, centroid, word_counting_top3_result):
    model_filename = OUTPUT_PATH + name + '_' + str(k) + '.model'
    model.save(model_filename)

    npoutfile = OUTPUT_PATH + name + '_' + str(k) + '_vector.npy'
    np.save(npoutfile, centroid)

    categoryoutfile = OUTPUT_PATH + name + '_' + str(k) + '_category.txt'
    category_df = pd.DataFrame(data = category)
    category_df.to_csv(categoryoutfile, index = False)

    keywordoutfile = OUTPUT_PATH + name + '_' + str(k) + '_top3keyword.txt'
    keyword_df = pd.DataFrame(word_counting_top3_result)
    keyword_df.to_csv(keywordoutfile, index = False)

for model in models:
    keyword_cluster_processing(model, "train")


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:119: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).


[[('조식', 7415), ('아침', 3283), ('식사', 3119)], [('호텔', 22858), ('위치', 4231), ('서비스', 3565)], [('체크', 7609), ('체크인', 4700), ('직원', 3229)], [('주차', 5369), ('주차장', 2088), ('불편', 1099)], [('칫솔', 1291), ('치약', 865), ('샴푸', 711)], [('방문', 3775), ('다음', 3639), ('이용', 2755)], [('침대', 4363), ('침구', 1308), ('편안', 757)], [('깨끗', 7045), ('깔끔', 5695), ('청결', 2774)], [('냄새', 3271), ('청소', 2576), ('먼지', 2321)], [('객실', 5291), ('호텔', 3317), ('침대', 2568)], [('에어컨', 1306), ('온도', 1224), ('난방', 1077)], [('소리', 3979), ('방음', 2129), ('소음', 1837)], [('가격', 9573), ('만족', 5072), ('위치', 4912)], [('위치', 10056), ('호텔', 4726), ('지하철', 4652)], [('직원', 15047), ('친절', 14171), ('서비스', 2535)], [('수영장', 3334), ('이용', 3214), ('시설', 920)], [('화장실', 3304), ('샤워', 2623), ('욕조', 1632)]]
[[('조식', 7415), ('아침', 3283), ('식사', 3119)], [('호텔', 22858), ('위치', 4231), ('서비스', 3565)], [('체크', 7609), ('체크인', 4700), ('직원', 3229)], [('주차', 5369), ('주차장', 2088), ('불편', 1099)], [('칫솔', 1291), ('치약', 865), ('샴푸', 711)], [('방문', 3775), ('다음'

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:119: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).


[[('소리', 4179), ('방음', 2310), ('소음', 1982)], [('호텔', 11058), ('체크', 7914), ('직원', 4857)], [('깨끗', 9399), ('호텔', 8561), ('가격', 8445)], [('직원', 16609), ('친절', 15340), ('서비스', 3332)], [('호텔', 11026), ('이용', 6213), ('방문', 4505)], [('조식', 8531), ('아침', 3885), ('식사', 3832)], [('화장실', 5394), ('침대', 5316), ('샤워', 3783)], [('위치', 12221), ('호텔', 8334), ('지하철', 5311)]]
[[('소리', 4179), ('방음', 2310), ('소음', 1982)], [('호텔', 11058), ('체크', 7914), ('직원', 4857)], [('깨끗', 9399), ('호텔', 8561), ('가격', 8445)], [('직원', 16609), ('친절', 15340), ('서비스', 3332)], [('호텔', 11026), ('이용', 6213), ('방문', 4505)], [('조식', 8531), ('아침', 3885), ('식사', 3832)], [('화장실', 5394), ('침대', 5316), ('샤워', 3783)], [('위치', 12221), ('호텔', 8334), ('지하철', 5311)]]
[[('소리', 4179), ('방음', 2310), ('소음', 1982)], [('체크', 7914), ('직원', 4857)], [('깨끗', 9399), ('호텔', 8561), ('가격', 8445)], [('직원', 16609), ('친절', 15340), ('서비스', 3332)], [('호텔', 11026), ('이용', 6213), ('방문', 4505)], [('조식', 8531), ('아침', 3885), ('식사', 3832)], [('화장실', 5394), ('침대', 

In [ ]:
def test_k_means_cluster(sentenceToken, preRawData, rawSentenceIdx, K):
    #x = np.array(sentenceToken)
    #x = torch.from_numpy(x)
    #cluster_ids_x, cluster_centers = kmeans(X=x, num_clusters=K, distance='euclidean', device=torch.device('cuda:0'))

    estimator = KMeans(n_clusters=K, init='k-means++', random_state=42, algorithm='elkan')
    estimator = estimator.fit(sentenceToken)
    cluster_ids_x = estimator.labels_
    cluster_centers = estimator.cluster_centers_

    #model = DBSCAN(min_samples=5000)
    #predict = pd.DataFrame(model.fit_predict(sentenceToken))
    #predict.columns = ['cluster']

    #print(predict)

    sentence_results = cluster_result_mapping(preRawData, cluster_ids_x, rawSentenceIdx, K)
    keyword_result = cluster_keyword_summarizer(sentence_results, K)

    return keyword_result, sentence_results, cluster_centers

def test_pre_keyword_cluster_processing(models):
    K = models[1]
    rawData, tokenData, rawSentenceIdx = data_load()
    if 'Word2Vec' in models[0]:
        embedding_model, word_vectors, vocabs, word_vectors_list = Word2Vec_Embedding(tokenData)
    else:
        embedding_model, word_vectors, vocabs, word_vectors_list = FastText_Embedding(tokenData)

    sentenceToken, preRawData, rawSentenceIdx = SentenceMapping(rawData, tokenData, word_vectors, rawSentenceIdx)

    return embedding_model, sentenceToken, preRawData, rawSentenceIdx, K

def test_keyword_cluster_processing(sentenceToken, preRawData, rawSentenceIdx, K):    
    k_cluster_keyword_result, k_cluster_sentence_result, cluster_centroid = test_k_means_cluster(sentenceToken, preRawData, rawSentenceIdx, K)
    return k_cluster_keyword_result, k_cluster_sentence_result, cluster_centroid

def ____(embedding_model, k_cluster_keyword_result, k_cluster_sentence_result):
    word_counting = cluster_based_word_counting_in_cluster(embedding_model, k_cluster_keyword_result, k_cluster_sentence_result)
    print(word_counting)
    category_name = []
    for cluster in word_counting:
        for item in cluster:
            if item == '호텔':
                continue
            if item == '칫솔' or item == '치약' or item == '수건' or item == '샴푸':
                item = '어메니티'
            category_name.append(item)
            break
    return category_name
    #save_model(models[0], K, embedding_model, category_name, cluster_centroid, word_counting)


In [ ]:
models = [['Word2Vec_test', 17], ['Word2Vec_cos', 8], ['FastText_cos', 17], ['FastText_cos', 8]]

embedding_model, sentenceToken, preRawData, rawSentenceIdx, K = test_pre_keyword_cluster_processing(models[0])

In [ ]:
k_cluster_keyword_result, k_cluster_sentence_result, cluster_centroid = test_keyword_cluster_processing(sentenceToken, preRawData, rawSentenceIdx, K)

In [ ]:
c = ____(embedding_model, k_cluster_keyword_result, k_cluster_sentence_result)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:117: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).


[['직원', '친절', '서비스'], ['이용', '방문', '다음'], ['호텔', '위치', '객실'], ['냄새', '청소', '먼지'], ['화장실', '샤워', '욕조'], ['조식', '아침', '식사'], ['가격', '만족', '대비'], ['에어컨', '온도', '난방'], ['깨끗', '깔끔', '객실'], ['체크', '체크인', '직원'], ['침대', '침구', '편안'], ['소리', '방음', '소음'], ['위치', '호텔', '지하철'], ['칫솔', '치약', '샴푸'], ['주차', '주차장', '불편']]
[['직원', '친절', '서비스'], ['이용', '방문', '다음'], ['호텔', '위치', '객실'], ['냄새', '청소', '먼지'], ['화장실', '샤워', '욕조'], ['조식', '아침', '식사'], ['가격', '만족', '대비'], ['에어컨', '온도', '난방'], ['깨끗', '깔끔', '객실'], ['체크', '체크인', '직원'], ['침대', '침구', '편안'], ['소리', '방음', '소음'], ['위치', '호텔', '지하철'], ['칫솔', '치약', '샴푸'], ['주차', '주차장', '불편']]
